In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import taxcalc as tc
from taxbrain import (
    TaxBrain, differences_plot, distribution_plot, volcano_plot
)
from bokeh.io import output_notebook, show

In [2]:
output_notebook()
pd.options.display.float_format = '{:,.3f}'.format

Loading BokehJS ...

In [3]:
# Set paths to work across Mac/Windows/Linux platforms
# Update cur_path to the path of the repsitory on your local machine
cur_path = "/Users/richardevans/Docs/Economics/OSE/Federal/CharitableContrib"
data_dir = os.path.join(cur_path, 'data')
images_dir = os.path.join(cur_path, 'images')
# Directories to save data
json_dir = os.path.join(cur_path, "json")
base_dir = os.path.join(cur_path, "OUTPUT_BASELINE")
reform_dir_chddct = os.path.join(cur_path, "OUTPUT_REFORM_CHDDCT")
tmd_dir = (
    "/Users/richardevans/Docs/Economics/OSE/microsim/" +
    "tax-microdata-benchmarking/tmd/storage/output"
)

## 2. Run Tax-Calculator analysis

### 2.1. Setup baseline calculator

In [4]:
# Set up the baseline calculator
base_pol = tc.policy.Policy()
base_pol_dict = base_pol.read_json_reform(os.path.join(json_dir, "ext.json"))
base_pol.implement_reform(base_pol_dict)
gfactors=tc.growfactors.GrowFactors(
    growfactors_filename=os.path.join(tmd_dir, "tmd_growfactors_jason.csv")
)
# rec = tc.records.Records.cps_constructor()
rec = tc.records.Records(
    data=os.path.join(tmd_dir, "tmd_jason.csv.gz"),
    start_year=tc.records.Records.TMDCSV_YEAR,
    gfactors=gfactors,
    weights=os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
    adjust_ratios=None,
)
tc_base = tc.calculator.Calculator(policy=base_pol, records=rec)
print("taxcalc TMD start year is:", tc.records.Records.TMDCSV_YEAR)


taxcalc TMD start year is: 2021


### 2.2. Setup reform calculator

In [5]:
ref_pol = tc.policy.Policy()
ref_pol.implement_reform(base_pol_dict)
ref_pol_dict = base_pol.read_json_reform(os.path.join(json_dir, "chddct.json"))
ref_pol.implement_reform(ref_pol_dict)
tc_ref = tc.calculator.Calculator(policy=ref_pol, records=rec)

### 2.3. Create the revenue estimates for 10-year budget window

In [6]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
start_year = 2025
end_year = 2034

rev_base = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
rev_ref = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_base.advance_to_year(t)
    tc_base.calc_all()
    rev_base['IIT'][t] = tc_base.weighted_total('iitax')
    rev_base['Payroll'][t] = tc_base.weighted_total('payrolltax')
    rev_base['Sum'][t] = rev_base['IIT'][t] + rev_base['Payroll'][t]
    tc_ref.advance_to_year(t)
    tc_ref.calc_all()
    rev_ref['IIT'][t] = tc_ref.weighted_total('iitax')
    rev_ref['Payroll'][t] = tc_ref.weighted_total('payrolltax')
    rev_ref['Sum'][t] = rev_ref['IIT'][t] + rev_ref['Payroll'][t]

### 2.4. Create a table of output

In [7]:
# make table of revenue estimates
rev_base_df = pd.DataFrame.from_dict(rev_base).T
rev_base_df['2025-2034'] = rev_base_df.sum(axis=1)
rev_ref_df = pd.DataFrame.from_dict(rev_ref).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff_df = (rev_ref_df - rev_base_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_df['2025-2034'] = diff_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_df * 1e-9)
print(rev_ref_df * 1e-9)
print(diff_df.to_markdown())

Charitable Act revenue estimates
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2696.091 $2811.543 $2933.517 $3065.789 $3203.963 $3348.031   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4287.415 $4481.680 $4670.579 $4863.570 $5069.763 $5284.271 $5506.526   

             2032      2033      2034  2025-2034  
IIT     $3497.398 $3649.704 $3805.891 $31592.409  
Payroll $2237.907 $2318.438 $2400.162 $20480.896  
Sum     $5735.305 $5968.141 $6206.053 $52073.305  
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2577.064 $2692.613 $2808.020 $2929.959 $3062.194 $3200.329 $3344.358   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4283.995 $4478.202 $4667.057 $4860.012 $5066.168 $5280.637 $5502.854   

             2032      2033      2034  
IIT     $3493.691 $3645.960 $3802.116  
Payroll $2237.907 $2318.438 $2400

## 3. Do the current Charitable Act $10,000/$5,000 for 2026 and 2027

### 3.1. Setup baseline calculator (same as previous)

### 3.2. Set up a reform calculator with the new policy

In [8]:
ref_pol_2 = tc.policy.Policy()
ref_pol_2.implement_reform(base_pol_dict)
ref_pol_2_dict = base_pol.read_json_reform(os.path.join(json_dir, "chddct2.json"))
ref_pol_2.implement_reform(ref_pol_2_dict)
tc_ref_2 = tc.calculator.Calculator(policy=ref_pol_2, records=rec)

### 3.3. Create the revenue estimates for 10-year budget window

In [9]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
# rev_base = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
rev_ref_2 = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_ref_2.advance_to_year(t)
    tc_ref_2.calc_all()
    rev_ref_2['IIT'][t] = tc_ref_2.weighted_total('iitax')
    rev_ref_2['Payroll'][t] = tc_ref_2.weighted_total('payrolltax')
    rev_ref_2['Sum'][t] = rev_ref_2['IIT'][t] + rev_ref_2['Payroll'][t]

### 3.4. Create a table of output

In [ ]:
# make table of revenue estimates
rev_ref_2_df = pd.DataFrame.from_dict(rev_ref_2).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff_2_df = (rev_ref_2_df - rev_base_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_2_df['2025-2034'] = diff_2_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_df * 1e-9)
print(rev_ref_2_df * 1e-9)
print(diff_2_df.to_markdown())

Charitable Act revenue estimates
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2696.091 $2811.543 $2933.517 $3065.789 $3203.963 $3348.031   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4287.415 $4481.680 $4670.579 $4863.570 $5069.763 $5284.271 $5506.526   

             2032      2033      2034  2025-2034  
IIT     $3497.398 $3649.704 $3805.891 $31592.409  
Payroll $2237.907 $2318.438 $2400.162 $20480.896  
Sum     $5735.305 $5968.141 $6206.053 $52073.305  
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $2680.299 $2795.304 $2933.517 $3065.789 $3203.963 $3348.031   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4287.415 $4465.888 $4654.341 $4863.570 $5069.763 $5284.271 $5506.526   

             2032      2033      2034  
IIT     $3497.398 $3649.704 $3805.891  
Payroll $2237.907 $2318.438 $2400

In [13]:
rev_ref_2_df.sum(axis=1) * 1e-9

IIT       $31560.379
Payroll   $20480.896
Sum       $52041.275
dtype: float64